In [1]:
import polars as pl

## Exploration and cleanup

In [2]:
category = pl.scan_csv('data/category.csv')

category.head().collect()

category_id,category_name
str,str
"""CAT-1""","""Laptop"""
"""CAT-2""","""Audio"""
"""CAT-3""","""Tablet"""
"""CAT-4""","""Smartphone"""
"""CAT-5""","""Wearable"""


In [3]:
products = (
    pl.scan_csv('data/products.csv', try_parse_dates=True)
    .rename(str.lower)
)

products.head().collect()

product_id,product_name,category_id,launch_date,price
str,str,str,date,i64
"""P-1""","""MacBook""","""CAT-1""",2023-09-17,1149
"""P-2""","""MacBook Air (M1)""","""CAT-1""",2023-11-11,1783
"""P-3""","""MacBook Air (M2)""","""CAT-1""",2020-05-24,1588
"""P-4""","""MacBook Pro 13-inch""","""CAT-1""",2021-01-17,1351
"""P-5""","""MacBook Pro 14-inch""","""CAT-1""",2024-05-12,768


In [4]:
stores = (
    pl.scan_csv('data/stores.csv')
    .rename(str.lower)
)

stores.head().collect()

store_id,store_name,city,country
str,str,str,str
"""ST-1""","""Apple Fifth Avenue""","""New York""","""United States"""
"""ST-2""","""Apple Union Square""","""San Francisco""","""United States"""
"""ST-3""","""Apple Michigan Avenue""","""Chicago""","""United States"""
"""ST-4""","""Apple The Grove""","""Los Angeles""","""United States"""
"""ST-5""","""Apple SoHo""","""New York""","""United States"""


In [5]:
sales = pl.scan_csv('data/sales.csv', try_parse_dates=True)

sales.head().collect()

sale_id,sale_date,store_id,product_id,quantity
str,date,str,str,i64
"""YG-8782""",2023-06-16,"""ST-10""","""P-38""",10
"""QX-999001""",2022-04-13,"""ST-63""","""P-48""",10
"""JG-46890""",2021-07-05,"""ST-26""","""P-79""",5
"""XJ-1731""",2022-07-20,"""ST-15""","""P-24""",9
"""FG-95080""",2022-03-18,"""ST-35""","""P-69""",7


In [6]:
warranty = pl.scan_csv('data/warranty.csv', try_parse_dates=True)

warranty.head().collect()

claim_id,claim_date,sale_id,repair_status
str,date,str,str
"""CL-58750""",2024-01-30,"""YG-8782""","""Completed"""
"""CL-8874""",2024-06-25,"""QX-999001""","""Pending"""
"""CL-14486""",2024-08-13,"""JG-46890""","""Pending"""
"""CL-42187""",2024-09-19,"""XJ-1731""","""Pending"""
"""CL-37590""",2024-09-16,"""FG-95080""","""Completed"""


## Descriptive analysis

### Revenue by country

In [7]:
agg_sales = (
    sales
    .join(products, how='left', on='product_id')
    .join(stores, how='left', on='store_id')
    .with_columns(
        (pl.col('quantity') * pl.col('price')).alias('revenue')
    )
    .group_by(
        pl.col('sale_date').dt.year().alias('year').cast(pl.UInt64),
        pl.col('country')
    )
    .agg(
        pl.len().alias('n_orders').cast(pl.UInt64),
        pl.col('quantity').sum().alias('units_sold').cast(pl.UInt64),
        pl.col('revenue').sum().alias('total_revenue').cast(pl.UInt64)
    )
    .sort(pl.col('year', 'country'))
).collect()

agg_sales

year,country,n_orders,units_sold,total_revenue
u64,str,u64,u64,u64
2020,"""Australia""",19990,110341,118359049
2020,"""Austria""",2790,15379,16846043
2020,"""Canada""",14377,78965,85102058
2020,"""China""",19967,110088,118359886
2020,"""Colombia""",5563,29967,32355688
…,…,…,…,…
2024,"""Taiwan""",2456,13593,14689981
2024,"""Thailand""",7353,40322,43487986
2024,"""UAE""",12153,66835,72172319


In [8]:
agg_sales.plot.line(x='year', y='total_revenue', color='country')

alt.Chart(...)

### Top products sold

In [9]:
top_products = (
    sales
    .join(products, on='product_id')
    .with_columns(
        (pl.col('quantity') * pl.col('price')).alias('revenue')
    )
    .group_by('product_id')
    .agg(
        pl.col('product_name').first(),
        pl.len().alias('n_orders').cast(pl.UInt64),
        pl.col('quantity').sum().alias('units_sold').cast(pl.UInt64),
        pl.col('revenue').sum().alias('total_revenue').cast(pl.UInt64),
    )
    .sort('total_revenue', descending=True)
).collect()

top_products.head(10)

product_id,product_name,n_orders,units_sold,total_revenue
str,str,u64,u64,u64
"""P-68""","""Apple Music""",11687,63844,125453460
"""P-58""","""iMac 27-inch""",11791,64654,124006372
"""P-25""","""iPad mini (5th Generation)""",11734,64828,123951136
"""P-23""","""iPad (9th Generation)""",11577,63597,123950553
"""P-17""","""Beats Fit Pro""",11706,64410,118449990
"""P-8""","""MacBook Air (Retina)""",11674,63947,118110109
"""P-12""","""AirPods (3rd Generation)""",11672,64102,118075884
"""P-31""","""iPad Pro (M2)""",11579,63543,116029518
"""P-28""","""iPad Pro 11-inch""",11703,64768,114056448


### Top categories sold

In [10]:
top_category = (
    top_products.lazy()
    .join(products, on='product_id', how='left')
    .join(category, on='category_id', how='left')
    .group_by('category_id')
    .agg(
        pl.col('category_name').first(),
        pl.col('n_orders', 'units_sold', 'total_revenue').sum(),
    )
    .sort('total_revenue', descending=True)
).collect()

top_category

category_id,category_name,n_orders,units_sold,total_revenue
str,str,u64,u64,u64
"""CAT-3""","""Tablet""",117142,645303,953443623
"""CAT-10""","""Accessories""",163849,899734,927115953
"""CAT-4""","""Smartphone""",151636,834006,865147932
"""CAT-2""","""Audio""",128731,707354,794980579
"""CAT-1""","""Laptop""",116369,640336,763382551
"""CAT-5""","""Wearable""",105521,582445,667537447
"""CAT-7""","""Desktop""",116812,642140,538481345
"""CAT-8""","""Subscription Service""",81561,447062,368463489
"""CAT-6""","""Streaming Device""",35014,192534,191622603


### Warranty claims rate

In [11]:
warranty_claim_rate = (
    warranty
    .join(sales, on='sale_id', how='left')
    .group_by('product_id')
    .agg(pl.len().alias('n_warranty_claims').cast(pl.UInt64))
    .join(top_products.lazy(), on='product_id', how='right')
    .with_columns(
        pl.col('n_warranty_claims').fill_null(0)
    )
    .with_columns(
        (pl.col('n_warranty_claims') / pl.col('n_orders')).alias('warranty_claim_rate')
    )
    .sort('warranty_claim_rate', descending=True)
).collect()

warranty_claim_rate

n_warranty_claims,product_id,product_name,n_orders,units_sold,total_revenue,warranty_claim_rate
u64,str,str,u64,u64,u64,f64
381,"""P-9""","""MacBook Pro (Touch Bar)""",11566,63691,83053064,0.032941
367,"""P-38""","""iPhone 13 Pro""",11551,63711,19622988,0.031772
370,"""P-17""","""Beats Fit Pro""",11706,64410,118449990,0.031608
368,"""P-69""","""Apple TV+""",11647,64103,25897612,0.031596
370,"""P-83""","""Smart Cover for iPad""",11808,64656,65561184,0.031335
…,…,…,…,…,…,…
304,"""P-54""","""Apple TV 4K""",11635,63975,80288625,0.026128
302,"""P-47""","""Apple Watch SE""",11614,64152,99178992,0.026003
302,"""P-70""","""Apple Fitness+""",11633,63733,58825559,0.025961


### Top sales over time

In [12]:
sales_over_time = (
    sales
    .group_by('sale_date', 'product_id')
    .agg(
        pl.col('quantity').sum().cast(pl.UInt64)
    )
    .join(top_products.lazy().head(2), on='product_id', how='right')
    .sort('sale_date')
    .select(
        pl.col('sale_date').alias('date'),
        (pl.col('product_id') + ': ' + pl.col('product_name')).alias('product'),
        pl.col('quantity').cum_sum()
    )
).collect()

sales_over_time.plot.line(x='date', y='quantity', color='product')

alt.Chart(...)